In [3]:
import pandas as pd
from plotnine import *

import ee
ee.Initialize(project='dri-apps')

### Initialize the webmap to select AOI

In [4]:
# Click a point on the map to get its coordinates
from ipyleaflet import Map

# Center the map on Nevada
center = (39.5, -117)  
m = Map(center=center, zoom=7)

# Define a callback to handle clicks and cast to ee.Geometry.Point
def handle_map_click(**kwargs):
    global coords_ee
    if kwargs.get('type') == 'click':
        coords = kwargs.get('coordinates')
        coords = [coords[1], coords[0]]
        print('You clicked on', coords)
        coords_ee = ee.Geometry.Point(coords)

# Attach the callback function to the map
m.on_interaction(handle_map_click)
m


Map(center=[39.5, -117], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out…

You clicked on [-116.98239924423707, 40.36182502269415]
You clicked on [-117.53156077911872, 40.887148958262905]


### Extract the data from Earth Engine

In [5]:
# ------------ Soils ------------

# Currently using texture at depth 25-50cm; deepest available and closest to water table depth
soil = ee.Image('projects/sat-io/open-datasets/CSRL_soil_properties/physical/soil_texture_profile/texture_2550').rename('texture')

# Define a dictionary that can be used as a lookup for the soil layer values using names consistent with CSV to filter
soil_lu_dict = ee.Dictionary({
    "1": "sand",
    "2": "loamysand",
    "3": "sandyloam",
    "4": "loam",
    "5": "siltloam",
    "6": "silt",
    "7": "sandyclayloam",
    "8": "clayloam",
    "9": "siltyclayloam",
    "10": "sandyclay",
    "11": "siltyclay",
    "12": "clay"})

# Extract soil texture for the point
soil_point = soil.reduceRegion(reducer = ee.Reducer.mean(), geometry = coords_ee, scale = 30).get('texture')
soil_string = soil_lu_dict.get(ee.Number(soil_point).format('%.0f')).getInfo()
print('The soil type is', soil_string)

# ----------- Water balance -----------

# Define years
year_start = 1991
year_end = 2023
year_list = ee.List.sequence(year_start, year_end)

# Calculate water year precip and ETo from gridMET
gm = ee.ImageCollection('IDAHO_EPSCOR/GRIDMET').select(['pr', 'eto'])
def calculate_wy_stats(year):
  date_start = ee.Date.fromYMD(ee.Number(year).subtract(1), 10, 1);
  date_end = ee.Date.fromYMD(ee.Number(year), 10, 1); # end date is exclusive so go past
  return ee.Image(gm.filterDate(date_start, date_end).sum()).set({
      'system:time_start': date_start.millis(),
      'year': ee.Number(year),
      'system:index': ee.Number(year).format('%.0f')
    })
gm_wy = ee.ImageCollection(year_list.map(calculate_wy_stats));

# Calculate the water balance and divide by 100
def calculate_wb(image):
  image_ws = image.select('pr').subtract(image.select('eto')).rename('wb')
  image_ws = image_ws.divide(100)
  return image.addBands(image_ws)
gm_wy = ee.ImageCollection(gm_wy).map(calculate_wb).toBands()

# Extract water balance for the point
gm_point = gm_wy.reduceRegion(reducer = ee.Reducer.mean(), geometry = coords_ee, scale = 4000).getInfo()
print(gm_point)

# Parse the dictionary into a DataFrame
parsed_data = []
for key, value in gm_point.items():
    year, suffix = key.split('_')
    parsed_data.append({'wy': int(year), suffix: value})

# Combine the parsed data into a DataFrame
dfee = pd.DataFrame(parsed_data)
dfee = dfee.groupby('wy').first().reset_index()

# Calculate annual water balance variables
dfee['wb2'] = dfee['wb'] ** 2  # Square of 'wb'
dfee['wb3'] = dfee['wb'] ** 3  # Cube of 'wb'

# Rename dfee as dfclimate
dfclimate = dfee
print('Climate timeseries')
print(dfclimate)

The soil type is clayloam
{'1991_eto': 1286.068045169115, '1991_pr': 266.501053661108, '1991_wb': -10.195669915080071, '1992_eto': 1399.46120133996, '1992_pr': 178.6473677456379, '1992_wb': -12.208138335943222, '1993_eto': 1238.7015956044197, '1993_pr': 318.2762913107872, '1993_wb': -9.204253042936324, '1994_eto': 1412.153010636568, '1994_pr': 204.31570586562157, '1994_wb': -12.078373047709466, '1995_eto': 1236.677718669176, '1995_pr': 390.517854899168, '1995_wb': -8.461598637700082, '1996_eto': 1359.902524754405, '1996_pr': 311.92712768912315, '1996_wb': -10.479753970652819, '1997_eto': 1307.8715744018555, '1997_pr': 369.16592705249786, '1997_wb': -9.387056473493576, '1998_eto': 1186.6845454871655, '1998_pr': 468.24072951078415, '1998_wb': -7.184438159763813, '1999_eto': 1320.6723357737064, '1999_pr': 268.1799817085266, '1999_wb': -10.524923540651798, '2000_eto': 1414.0601915419102, '2000_pr': 258.54915142059326, '2000_wb': -11.555110401213168, '2001_eto': 1360.2267215251923, '2001_pr

### Import model coefficients, join to water balance data from EE, and calculate LAI, AET, GWSUBS, etc.

In [7]:
# Load the dataset
dfcoeffs = pd.read_csv('../data/MixedEffectsModelCoefficients102924_LAI_AET_AETG_GWsubs.csv')
dfcoeffs['wtd2'] = dfcoeffs['WTD'].apply(lambda x: "Free Drain" if x == 12 else f"{x} m")
# print(dfcoeffs)

# Define rooting depth and soil type
rd = 0.5 # TODO - get from user input
st = soil_string # TODO - get from user input (user could override soil type)

# Filter the coefficients for the soil type and rooting depth
dfcoeffs = dfcoeffs[dfcoeffs['rootdepth'] == rd]
dfcoeffs = dfcoeffs[dfcoeffs['soiltype'] == soil_string]

# Add WTD column to dfclimate
wtd_values = [1, 3, 6, 12]
dfclimate = pd.concat([dfclimate.assign(WTD=wtd) for wtd in wtd_values], ignore_index=True)

# Left join dfclimate and dfcoeffs 
dfsum = pd.merge(dfclimate, dfcoeffs, left_on='WTD', right_on='WTD', how='left')

# Calculate LAI, AET, AETgw, and GWsubs
dfsum['LAIcalc'] = (
    dfsum['LAIIntercept'] +
    dfsum['wb'] * dfsum['LAIwbx'] +
    dfsum['wb2'] * dfsum['LAIwb2x'] +
    dfsum['wb3'] * dfsum['LAIwb3x']
)
dfsum['aetcalc'] = (
    dfsum['aetIntercept'] +
    dfsum['wb'] * dfsum['aetwbx'] +
    dfsum['wb2'] * dfsum['aetwb2x'] +
    dfsum['wb3'] * dfsum['aetwb3x']
)
dfsum['aetgwcalc'] = (
    dfsum['aetgwIntercept'] +
    dfsum['wb'] * dfsum['aetgwwbx'] +
    dfsum['wb2'] * dfsum['aetgwwb2x'] +
    dfsum['wb3'] * dfsum['aetgwwb3x']
)
dfsum['gwsubscalc'] = (
    dfsum['gwsubsIntercept'] +
    dfsum['wb'] * dfsum['gwsubswbx'] +
    dfsum['wb2'] * dfsum['gwsubswb2x'] +
    dfsum['wb3'] * dfsum['gwsubswb3x']
)

dfsum

,wy,eto,pr,wb,wb2,wb3,WTD,allcats,soiltype,rootdepth,...,aetgwwb3x,gwsubsIntercept,gwsubswbx,gwsubswb2x,gwsubswb3x,wtd2,LAIcalc,aetcalc,aetgwcalc,gwsubscalc
0,1991,1286.068045,266.501054,-10.195670,103.951685,-1059.857068,1,clayloam_0.5_1,clayloam,0.5,...,0.004257,-21.182581,-39.357377,0.123825,0.014034,1 m,1.218444,470.238839,-0.102334,378.090246
1,1992,1399.461201,178.647368,-12.208138,149.038642,-1819.484354,1,clayloam_0.5_1,clayloam,0.5,...,0.004257,-21.182581,-39.357377,0.123825,0.014034,1 m,1.300504,510.939926,-0.075935,452.218154
2,1993,1238.701596,318.276291,-9.204253,84.718274,-779.768432,1,clayloam_0.5_1,clayloam,0.5,...,0.004257,-21.182581,-39.357377,0.123825,0.014034,1 m,1.181817,453.743672,-0.001545,340.619811
3,1994,1412.153011,204.315706,-12.078373,145.887095,-1762.078762,1,clayloam_0.5_1,clayloam,0.5,...,0.004257,-21.182581,-39.357377,0.123825,0.014034,1 m,1.294967,508.025691,-0.083983,447.526311
4,1995,1236.677719,390.517855,-8.461599,71.598652,-605.839052,1,clayloam_0.5_1,clayloam,0.5,...,0.004257,-21.182581,-39.357377,0.123825,0.014034,1 m,1.156472,442.937993,0.143703,312.207235
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,2019,1242.900001,537.799999,-7.051000,49.716601,-350.551757,12,clayloam_0.5_12,clayloam,0.5,...,0.001956,NaN,NaN,NaN,NaN,Free Drain,0.610548,166.747217,-0.113184,NaN
128,2020,1392.200001,197.499999,-11.947000,142.730810,-1705.204985,12,clayloam_0.5_12,clayloam,0.5,...,0.001956,NaN,NaN,NaN,NaN,Free Drain,0.326493,63.498468,-0.031302,NaN
129,2021,1391.000000,238.899999,-11.521000,132.733441,-1529.221977,12,clayloam_0.5_12,clayloam,0.5,...,0.001956,NaN,NaN,NaN,NaN,Free Drain,0.345885,69.908288,-0.060719,NaN
130,2022,1304.100000,291.200001,-10.129000,102.596641,-1039.201374,12,clayloam_0.5_12,clayloam,0.5,...,0.001956,NaN,NaN,NaN,NaN,Free Drain,0.418033,94.432995,-0.142455,NaN


### Read-in the Mixed-effects Model CSV and join the water balance data from EE
This is no longer needed now that we're calculating all variables on the fly

In [ ]:
# # Load the dataset
# dfsum = pd.read_csv('MixedEffectsModel_filtered_results_102924_clean.csv')

# # Ensure the types of 'wy' and 'Year' columns match
# ee_df['wy'] = ee_df['wy'].astype(int)  # Convert 'Year' to int

# # Perform the merge again
# dfsum = pd.merge(dfsum, ee_df, left_on='wy', right_on='wy', how='left')
# dfsum['wb'] = dfsum['wb_y']

# # Drop unnecessary columns
# columns_to_drop = ['wb2', 'wb3', 'PotET', 'Precip', 'wb_x', 'wb_y']
# dfsum = dfsum.drop(columns=columns_to_drop, errors='ignore')
# print(dfsum)

### Generate the Leaf Area Index figures

In [9]:
# Set threshold based on root depth
laithresh = 1.5 if rd == 0.5 else (2 if rd == 2 else 1 if rd == 3.6 else None)

# Group by wtd2 and count rows where LAIcalc >= laithresh
filtered_lai = dfsum[dfsum["LAIcalc"] >= laithresh]
laisum = (
    filtered_lai
    .groupby("wtd2")
    .size()  # counts the number of rows per group
    .reset_index(name="noverthresh")
)

# Left-join lai with laisum
lai2 = pd.merge(dfsum, laisum, on="wtd2", how="left")

# Replace NaN values in 'noverthresh' with 0
lai2["noverthresh"] = lai2["noverthresh"].fillna(0)

# Calculate the percentage (percoverthresh) and round
lai2["percoverthresh"] = round(lai2["noverthresh"] / 21, 2) * 100 # Todo: remove the rounding
lai2

,wy,eto,pr,wb,wb2,wb3,WTD,allcats,soiltype,rootdepth,...,gwsubswbx,gwsubswb2x,gwsubswb3x,wtd2,LAIcalc,aetcalc,aetgwcalc,gwsubscalc,noverthresh,percoverthresh
0,1991,1286.068045,266.501054,-10.195670,103.951685,-1059.857068,1,clayloam_0.5_1,clayloam,0.5,...,-39.357377,0.123825,0.014034,1 m,1.218444,470.238839,-0.102334,378.090246,0.0,0.0
1,1992,1399.461201,178.647368,-12.208138,149.038642,-1819.484354,1,clayloam_0.5_1,clayloam,0.5,...,-39.357377,0.123825,0.014034,1 m,1.300504,510.939926,-0.075935,452.218154,0.0,0.0
2,1993,1238.701596,318.276291,-9.204253,84.718274,-779.768432,1,clayloam_0.5_1,clayloam,0.5,...,-39.357377,0.123825,0.014034,1 m,1.181817,453.743672,-0.001545,340.619811,0.0,0.0
3,1994,1412.153011,204.315706,-12.078373,145.887095,-1762.078762,1,clayloam_0.5_1,clayloam,0.5,...,-39.357377,0.123825,0.014034,1 m,1.294967,508.025691,-0.083983,447.526311,0.0,0.0
4,1995,1236.677719,390.517855,-8.461599,71.598652,-605.839052,1,clayloam_0.5_1,clayloam,0.5,...,-39.357377,0.123825,0.014034,1 m,1.156472,442.937993,0.143703,312.207235,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,2019,1242.900001,537.799999,-7.051000,49.716601,-350.551757,12,clayloam_0.5_12,clayloam,0.5,...,NaN,NaN,NaN,Free Drain,0.610548,166.747217,-0.113184,NaN,0.0,0.0
128,2020,1392.200001,197.499999,-11.947000,142.730810,-1705.204985,12,clayloam_0.5_12,clayloam,0.5,...,NaN,NaN,NaN,Free Drain,0.326493,63.498468,-0.031302,NaN,0.0,0.0
129,2021,1391.000000,238.899999,-11.521000,132.733441,-1529.221977,12,clayloam_0.5_12,clayloam,0.5,...,NaN,NaN,NaN,Free Drain,0.345885,69.908288,-0.060719,NaN,0.0,0.0
130,2022,1304.100000,291.200001,-10.129000,102.596641,-1039.201374,12,clayloam_0.5_12,clayloam,0.5,...,NaN,NaN,NaN,Free Drain,0.418033,94.432995,-0.142455,NaN,0.0,0.0


In [10]:
# Plot LAI time series
p_lai1 = (
    ggplot(lai2) +
    geom_line(aes('wy', 'LAIcalc', linetype='wtd2')) +
    geom_point(aes('wy', 'LAIcalc', color='wb * 100')) +
    geom_hline(yintercept=laithresh, alpha=0.5) +
    geom_text(aes(x=2007, y=laithresh + 0.1), label=f"Example Management Target, LAI={laithresh}") +
    theme_bw() +
    scale_color_distiller(palette="YlGnBu", direction=1) +
    ggtitle(f"Root Depth: {rd}m, Soil Type: {st}") +
    labs(x="Water Year", y="Annual Maximum Leaf Area Index (LAI)", color="Annual Potential\nWater Deficit (mm)", linetype="Water Table Depth")
)
p_lai1.save(f"../outputs/python/{st}_{rd}_rootdepth_timeseries_LAI.png", height=4, width=6, units="in", dpi=300)

# Plot LAI boxplot
p_lai2 = (
    ggplot(lai2) +
    geom_boxplot(aes('wtd2', 'LAIcalc')) +
    geom_point(aes('wtd2', 'LAIcalc', color='wb * 100')) +
    geom_hline(yintercept=laithresh, alpha=0.5) +
    geom_text(aes(x='wtd2', y=laithresh, label='percoverthresh'), va="bottom") +
    theme_bw() +
    scale_color_distiller(palette="YlGnBu", direction=1) +
    ggtitle(f"Root Depth: {rd}m, Soil Type: {st}") +
    labs(x="Water Table Depth", y="Annual Maximum Leaf Area Index (LAI)", color="Annual Potential\nWater Deficit (mm)")
)
p_lai2.save(f"../outputs/python/{st}_{rd}_rootdepth_boxplot_LAI.png", height=4, width=6, units="in", dpi=300)

/opt/homebrew/lib/python3.11/site-packages/plotnine/ggplot.py:587: PlotnineWarning: Saving 6 x 4 in image.
/opt/homebrew/lib/python3.11/site-packages/plotnine/ggplot.py:588: PlotnineWarning: Filename: ../outputs/python/clayloam_0.5_rootdepth_timeseries_LAI.png
/opt/homebrew/lib/python3.11/site-packages/plotnine/ggplot.py:587: PlotnineWarning: Saving 6 x 4 in image.
/opt/homebrew/lib/python3.11/site-packages/plotnine/ggplot.py:588: PlotnineWarning: Filename: ../outputs/python/clayloam_0.5_rootdepth_boxplot_LAI.png


### Generate the AET figures

In [11]:
# Group by wtd2 and calculate min, max of aetcalc
aetsum = (
    dfsum.groupby("wtd2")
    .agg(min=("aetcalc", "min"), max=("aetcalc", "max"))
    .reset_index()
)

# Left-join aet with aetsum
aet2 = pd.merge(dfsum, aetsum, on="wtd2", how="left")

# Round min and max to 0 digits
aet2["min"] = aet2["min"].round(0)
aet2["max"] = aet2["max"].round(0)

# Create a rangelab column (e.g., "10-25")
aet2["rangelab"] = aet2["min"].astype(str) + "-" + aet2["max"].astype(str)

# charttop corresponds to the 'max' column in the DataFrame
charttop = aet2["max"]

In [12]:
# Plot AET time series
p_aet1 = (
    ggplot(aet2) +
    geom_line(aes('wy', 'aetcalc', linetype='wtd2')) +
    geom_point(aes('wy', 'aetcalc', color='wb * 100')) +
    theme_bw() +
    scale_color_distiller(palette="YlGnBu", direction=1) +
    ggtitle(f"Root Depth: {rd}m, Soil Type: {st}") +
    labs(x="Water Year", y="Annual Actual Evapotranspiration-Total (mm)", color="Annual Potential\nWater Deficit (mm)", linetype="Water Table Depth")
)
p_aet1.save(f"../outputs/python/{st}_{rd}_rootdepth_timeseries_totalET.png", height=4, width=6, units="in", dpi=300)

# Plot AET boxplot
p_aet2 = (
    ggplot(aet2) +
    geom_boxplot(aes('wtd2', 'aetcalc')) +
    geom_point(aes('wtd2', 'aetcalc', color='wb * 100')) +
    geom_text(aes(x='wtd2', y='max', label='rangelab'), va="bottom") +
    theme_bw() +
    scale_color_distiller(palette="YlGnBu", direction=1) +
    ggtitle(f"Root Depth: {rd}m, Soil Type: {st}") +
    labs(x="Water Table Depth", y="Annual Actual Evapotranspiration-Total (mm)",  color="Annual Potential\nWater Deficit (mm)")
)
p_aet2.save(f"../outputs/python/{st}_{rd}_rootdepth_boxplot_totalET.png", height=4, width=6, units="in", dpi=300)

/opt/homebrew/lib/python3.11/site-packages/plotnine/ggplot.py:587: PlotnineWarning: Saving 6 x 4 in image.
/opt/homebrew/lib/python3.11/site-packages/plotnine/ggplot.py:588: PlotnineWarning: Filename: ../outputs/python/clayloam_0.5_rootdepth_timeseries_totalET.png
/opt/homebrew/lib/python3.11/site-packages/plotnine/ggplot.py:587: PlotnineWarning: Saving 6 x 4 in image.
/opt/homebrew/lib/python3.11/site-packages/plotnine/ggplot.py:588: PlotnineWarning: Filename: ../outputs/python/clayloam_0.5_rootdepth_boxplot_totalET.png


### Generate the Groundwater subsidy figures

In [13]:
# Filter the DataFrame
gwsubs = dfsum[
    (dfsum["wtd2"] != "Free Drain")
]

# Create 'ratio' column for gwsubscalc/aetcalc
gwsubs = gwsubs.assign(ratio = gwsubs["gwsubscalc"] / gwsubs["aetcalc"])

# Group by wtd2 and compute min, max, minperc, maxperc
gwsubssum = (
    gwsubs.groupby("wtd2")
    .agg(
        min=("gwsubscalc", "min"),
        max=("gwsubscalc", "max"),
        minperc=("ratio", "min"),   # min of (gwsubscalc / aetcalc)
        maxperc=("ratio", "max"),   # max of (gwsubscalc / aetcalc)
    )
    .reset_index()
)

# Left-join the summarized data back to the filtered data
gwsubs2 = pd.merge(gwsubs, gwsubssum, on="wtd2", how="left")

# Round min and max to 0 decimals
gwsubs2["min"] = gwsubs2["min"].round(0)
gwsubs2["max"] = gwsubs2["max"].round(0)

# Create a range label (e.g., "10-25")
gwsubs2["rangelab"] = gwsubs2["min"].astype(str) + "-" + gwsubs2["max"].astype(str)

# Define charttop as the 'max' column
charttop = gwsubs2["max"]

# Multiply minperc, maxperc by 100 and round
gwsubs2["minperc"] = (gwsubs2["minperc"] * 100).round(0)
gwsubs2["maxperc"] = (gwsubs2["maxperc"] * 100).round(0)

# Create a percentage range label (e.g., "10-30% of Actual ET")
gwsubs2["rangelabperc"] = (
    gwsubs2["minperc"].astype(str)
    + "-"
    + gwsubs2["maxperc"].astype(str)
    + "% of Actual ET"
)

In [14]:
# Plot GWsubs time series
p_gwsubs1 = (
    ggplot(gwsubs2) +
    geom_line(aes('wy', 'gwsubscalc', linetype='wtd2')) +
    geom_point(aes('wy', 'gwsubscalc', color='gwsubscalc')) +
    theme_bw() +
    scale_color_distiller(palette="YlGnBu", direction=1) +
    ggtitle(f"Root Depth: {rd}m, Soil Type: {st}") +
    labs(x="Water Year", y="Annual Groundwater Subsidy (mm)", color="Annual Potential\nWater Deficit (mm)", linetype="Water Table Depth")
)
p_gwsubs1.save(f"../outputs/python/{st}_{rd}_rootdepth_timeseries_GWsubsET.png", height=4, width=6, units="in", dpi=300)

# Plot GWsubs boxplot
p_gwsubs2 = (
    ggplot(gwsubs2) +
    geom_boxplot(aes('wtd2', 'gwsubscalc')) +
    geom_point(aes('wtd2', 'gwsubscalc', color='gwsubscalc')) +
    geom_text(aes(x='wtd2', y='max', label='rangelabperc'), va="bottom") +
    theme_bw() +
    scale_color_distiller(palette="YlGnBu", direction=1) +
    ggtitle(f"Root Depth: {rd}m, Soil Type: {st}") +
    labs(x="Water Table Depth", y="Annual Groundwater Subsidy (mm)",  color="Annual Potential\nWater Deficit (mm)")
)
p_gwsubs2.save(f"../outputs/python/{st}_{rd}_rootdepth_boxplot_GWsubsET.png", height=4, width=6, units="in", dpi=300)

/opt/homebrew/lib/python3.11/site-packages/plotnine/ggplot.py:587: PlotnineWarning: Saving 6 x 4 in image.
/opt/homebrew/lib/python3.11/site-packages/plotnine/ggplot.py:588: PlotnineWarning: Filename: ../outputs/python/clayloam_0.5_rootdepth_timeseries_GWsubsET.png
/opt/homebrew/lib/python3.11/site-packages/plotnine/ggplot.py:587: PlotnineWarning: Saving 6 x 4 in image.
/opt/homebrew/lib/python3.11/site-packages/plotnine/ggplot.py:588: PlotnineWarning: Filename: ../outputs/python/clayloam_0.5_rootdepth_boxplot_GWsubsET.png
